In [1]:
import h5py
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, GlobalAveragePooling1D, Dense
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [2]:
# Load h5 data
with h5py.File('data\\train_df_30.h5', 'r') as f:
    x_train = f['x'][:]
    y_train = f['y'][:]
with h5py.File('data\\validation_df_30.h5', 'r') as f:
    x_validation = f['x'][:]
    y_validation = f['y'][:]
with h5py.File('data\\test_unit_13_30.h5', 'r') as f:
    x_test_13 = f['x'][:]
    y_test_13 = f['y'][:]
with h5py.File('data\\test_unit_14_30.h5', 'r') as f:
    x_test_14 = f['x'][:]
    y_test_14 = f['y'][:]
with h5py.File('data\\test_unit_15_30.h5', 'r') as f:
    x_test_15 = f['x'][:]
    y_test_15 = f['y'][:]

In [3]:
# Convert to tf tensors
X_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_validation = tf.convert_to_tensor(x_validation, dtype=tf.float32)
y_validation = tf.convert_to_tensor(y_validation, dtype=tf.float32)

In [4]:
# Separate into y labels (RUL and HS)
y_train = [tf.convert_to_tensor(y_train[:,0], dtype=tf.float32), 
           tf.convert_to_tensor(y_train[:,1], dtype=tf.float32)]
y_validation = [tf.convert_to_tensor(y_validation[:,0], dtype=tf.float32), 
                tf.convert_to_tensor(y_validation[:,1], dtype=tf.float32)]

In [5]:
# Check shape of data
print(X_train.shape)
print(y_train[0])
print(y_train[1])
print(X_validation.shape)
print(y_validation[0])
print(y_validation[1])

(1028672, 30, 18)
tf.Tensor([45.  8. 21. ... 44. 59. 54.], shape=(1028672,), dtype=float32)
tf.Tensor([0. 0. 0. ... 1. 1. 1.], shape=(1028672,), dtype=float32)
(114297, 30, 18)
tf.Tensor([53. 32. 53. ... 36. 63. 60.], shape=(114297,), dtype=float32)
tf.Tensor([1. 0. 0. ... 0. 1. 1.], shape=(114297,), dtype=float32)


In [6]:
# Check min, max, and average of y labels
print("Y_train RUL Cycles Min:", tf.reduce_min(y_train[0]).numpy())
print("Y_train RUL Cycles Max:", tf.reduce_max(y_train[0]).numpy())
print("Y_train Health State Min:", tf.reduce_min(y_train[1]).numpy())
print("Y_train Health State Max:", tf.reduce_max(y_train[1]).numpy())
print("Y_train RUL Cycles Average:", tf.reduce_mean(y_train[0]).numpy())
print("Y_train Health State Average:", tf.reduce_mean(y_train[1]).numpy())

Y_train RUL Cycles Min: 0.0
Y_train RUL Cycles Max: 92.0
Y_train Health State Min: 0.0
Y_train Health State Max: 1.0
Y_train RUL Cycles Average: 34.2329
Y_train Health State Average: 0.3089634


In [7]:
TIME_STEPS = 30
BATCH_SIZE = 64
steps_per_epoch = int(len(X_train)// BATCH_SIZE // 15)
validation_steps = int(len(X_validation)// BATCH_SIZE)
EPOCHS = 500
print(steps_per_epoch)
print(validation_steps)

1071
1785


In [8]:
def create_rul_generator(X, y, batch_size):
    """
    Generator function to yield batches of input features and RUL labels.

    Args:
        X (tf.Tensor): Input features tensor.
        y (list): List containing tensors for labels, where y[0] corresponds to RUL labels.
        batch_size (int): Number of samples per batch.

    Yields:
        tuple: A tuple containing a batch of input features and the corresponding RUL labels.
    """
    while True:
        for i in range(0, len(X), batch_size):
            yield X[i:i+batch_size], y[0][i:i+batch_size]

train_rul_generator = create_rul_generator(X_train, y_train, BATCH_SIZE)
validation_rul_generator = create_rul_generator(X_validation, y_validation, BATCH_SIZE)

def create_hs_generator(X, y, batch_size):
    """
    Generator function to yield batches of input features and health state labels.

    Args:
        X (tf.Tensor): Input features tensor.
        y (list): List containing tensors for labels, where y[1] corresponds to health state labels.
        batch_size (int): Number of samples per batch.

    Yields:
        tuple: A tuple containing a batch of input features and the corresponding health state labels.
    """
    while True:
        for i in range(0, len(X), batch_size):
            yield X[i:i+batch_size], y[1][i:i+batch_size]

train_hs_generator = create_hs_generator(X_train, y_train, BATCH_SIZE)
validation_hs_generator = create_hs_generator(X_validation, y_validation, BATCH_SIZE)

In [9]:
# Define input shape
input_shape = (TIME_STEPS, x_train.shape[2])

# Create input layer
inputs = Input(shape=input_shape, name='Input_Layer')

In [10]:
def create_hs_model(inputs):
    # Conv1D Block
    x = Conv1D(filters=512, kernel_size=3, strides=1, activation='relu',
                               padding='same', name='Conv1D_Layer1', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling1D()(x)

    # First Dense Block
    x = Dense(units=2048, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.025))(x)
    x = BatchNormalization()(x)

    # Following Dense Blocks
    for i in range(7):
      x = Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.025))(x)
      x = BatchNormalization()(x)

    # Output Layer
    health_state = Dense(1, activation='sigmoid', name='Health_State')(x)

    # Define the model
    model = Model(inputs=inputs, outputs=[health_state])
    
    # Define learning rate schedule
    lr_schedule = ExponentialDecay(initial_learning_rate=.0005, decay_steps=steps_per_epoch, decay_rate=.78)
    
    # Compile the model with a lower learning rate
    optimizer = AdamW(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer,
                  loss={'Health_State': 'binary_crossentropy'},
                  metrics={'Health_State': 'accuracy'})

    return model

hs_model = create_hs_model(inputs)

In [11]:
def custom_loss(y_true, y_pred):
    penalty_weight = .05
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    penalty = tf.reduce_mean(tf.square(tf.maximum(y_pred - y_true, 0)))
    total_loss = mse + penalty_weight * penalty
    return total_loss

def create_rul_model(inputs):
    # Conv1D Block
    x = Conv1D(filters=512, kernel_size=3, strides=1, activation='relu',
                               padding='same', name='Conv1D_Layer1', use_bias=False)(inputs)
    x = BatchNormalization(name='BatchNorm_Layer1')(x)
    x = GlobalAveragePooling1D()(x)

    # First Dense Block
    x = Dense(units=2048, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.025))(x)
    x = BatchNormalization()(x)

    # Following Dense Blocks
    for i in range(7):
      x = Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.025))(x)
      x = BatchNormalization()(x)

    # Output Layer
    rul_cycles = Dense(1, activation='linear', name='RUL_Cycles')(x)

    # Define the model
    model = Model(inputs=inputs, outputs=[rul_cycles])

    # Define learning rate schedule
    lr_schedule = ExponentialDecay(initial_learning_rate=.0007, decay_steps=steps_per_epoch, decay_rate=.78)
    
    # Compile the model with a lower learning rate
    optimizer = AdamW(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer,
                  loss={'RUL_Cycles': custom_loss},
                  metrics={'RUL_Cycles': 'mae'})

    return model

rul_model = create_rul_model(inputs)

In [12]:
hs_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_Layer (InputLayer)        │ (None, 30, 18)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1D_Layer1 (Conv1D)          │ (None, 30, 512)        │        27,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2048)           │     1,050,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 1,453,569 (5.54 MB)

 Trainable params: 1,446,657 (5.52 MB)

 Non-trainable params: 6,912 (27.00 KB)

In [ ]:
# Define early stopping
early_hs_stopping = EarlyStopping(monitor='val_accuracy', patience=30, mode='max', restore_best_weights=True)

# Train HS model
hs_history = hs_model.fit(train_hs_generator,
                    validation_data=validation_hs_generator,
                    epochs=EPOCHS,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    callbacks=[early_hs_stopping])

In [ ]:
rul_model.summary()

In [ ]:
# Define early stopping
early_rul_stopping = EarlyStopping(monitor='val_mae', patience=30, mode='min', restore_best_weights=True)

# Train RUL model
rul_history = rul_model.fit(train_rul_generator,
                    validation_data=validation_rul_generator,
                    epochs=EPOCHS,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    callbacks=[early_rul_stopping])

In [ ]:
# Print best
print("RUL MAE: ", min(rul_history.history['val_mae']))
print("HS Acc: ", max(hs_history.history['val_accuracy']))

In [15]:
# Save models
hs_model.save('hs_nn_model.keras')
rul_model.save('rul_nn_model.keras')

In [ ]:
# Create feature extractors
hs_feature_extractor = Model(inputs=hs_model.input, outputs=hs_model.layers[3].output)
rul_feature_extractor = Model(inputs=rul_model.input, outputs=rul_model.layers[3].output)

In [ ]:
# Create feature extracted datasets
hs_x_train_conv = hs_feature_extractor.predict(X_train)
hs_x_validation_conv = hs_feature_extractor.predict(X_validation)
hs_x_test_13_conv = hs_feature_extractor.predict(x_test_13)
hs_x_test_14_conv = hs_feature_extractor.predict(x_test_14)
hs_x_test_15_conv = hs_feature_extractor.predict(x_test_15)

rul_x_train_conv = rul_feature_extractor.predict(X_train)
rul_x_validation_conv = rul_feature_extractor.predict(X_validation)
rul_x_test_13_conv = rul_feature_extractor.predict(x_test_13)
rul_x_test_14_conv = rul_feature_extractor.predict(x_test_14)
rul_x_test_15_conv = rul_feature_extractor.predict(x_test_15)

In [ ]:
# Convert to pd dataframes and save as h5
hs_x_train_conv = pd.DataFrame(hs_x_train_conv)
hs_x_validation_conv = pd.DataFrame(hs_x_validation_conv)
hs_x_test_13_conv = pd.DataFrame(hs_x_test_13_conv)
hs_x_test_14_conv = pd.DataFrame(hs_x_test_14_conv)
hs_x_test_15_conv = pd.DataFrame(hs_x_test_15_conv)

rul_x_train_conv = pd.DataFrame(rul_x_train_conv)
rul_x_validation_conv = pd.DataFrame(rul_x_validation_conv)
rul_x_test_13_conv = pd.DataFrame(rul_x_test_13_conv)
rul_x_test_14_conv = pd.DataFrame(rul_x_test_14_conv)
rul_x_test_15_conv = pd.DataFrame(rul_x_test_15_conv)


In [ ]:
# Save feature extracted datasets


In [ ]:
# Save feature extractors
hs_feature_extractor.save('hs_feature_extractor.keras')
rul_feature_extractor.save('rul_feature_extractor.keras')